In [1]:
pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 237 kB 57.6 MB/s 
     |████████████████████████████████| 51 kB 8.8 MB/s 
     |████████████████████████████████| 5.8 MB 75.8 MB/s 
     |████████████████████████████████| 1.3 MB 64.9 MB/s 
     |████████████████████████████████| 182 kB 78.3 MB/s 
     |████████████████████████████████| 7.6 MB 71.6 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=d41e5514eed532d54a43b8724c5ea2e567f7a0af993f1e2a23ce9de93e2ef06a
  Stored in directory: /root/.cache/pip/wheels/6c/bc/8b/a51bee77aec33895e6c8c236144b4cc10875659c4d2c80f070
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3e852ded435e3f25432cb09b493be67a2fb51074bd2e9066a04464c4510e9a58
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import gc
import json
import errno
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join as pjoin
from glob import iglob
from shutil import rmtree
from tqdm import tqdm

# 데이터 불러오기

In [4]:
PATH = "/content/drive/MyDrive/ChatBot/chat_data/"

train_df = pd.read_parquet(PATH + "train.parquet")
valid_df = pd.read_parquet(PATH + "valid.parquet")
test_df = pd.read_parquet(PATH + "test.parquet")

In [5]:
data = pd.concat([train_df, valid_df, test_df], ignore_index=True)

In [10]:
train_df

,type,sess_id,speaker_1_id,query,speaker_2_id,reply
0,train,2-42648-1,30284.0,안녕하세요. 저는 30대 남성입니다.,34716.0,"안녕하세요, 저도 30대 남성입니다. 반갑습니다."
1,train,2-42648-1,30284.0,반가워요 저는 선박정비원으로 일하고있습니다. 혹시 직업이 어떻게되시나요?,34716.0,선박정비원이라니 멋지시네요. 저는 군사학과를 전공하고 지금 직업을 구하는 중입니다.
2,train,2-42648-1,30284.0,군사학과는 주로 무슨 직업으로 가는건가요?,34716.0,"주로 군인이 많이 되는데요, 특수 보직을 맡아 가기도 합니다. <u>저는 운전하는 ..."
3,train,2-42648-1,30284.0,저는 군대 때 운전병으로 나왔는데 운전 보직 좋은 것 같아요.,34716.0,그렇군요. 반갑습니다. 저도 하는일이 힘들긴 하지만 가끔 만족스럽습니다.<u>혹시 ...
4,train,2-42648-1,30284.0,저는 삼겹살을 좋아합니다. 점심시간인데 점심은 드셧나요?,34716.0,"네, 그렇게 좋아하는 토마토 파스타를 먹었네요. 요즘 일교차가 커서 그런지 감기 환..."
...,...,...,...,...,...,...
141184,train,2-17419-2,11285.0,아직 안갔습니다 10월 5일부터 10월14일까지 개최된다고 하여 저는 마지막날 가보...,32568.0,저는 예매를 하지 않아서 이번에도 못갈것 같습니다. 어떤 영화예매하셨나요?
141185,train,2-17419-2,11285.0,피터 본 카트 입니다. 영화제가 부산말고도 종류가 많은 것 같더라구요,32568.0,오 어디있나요? 저는 부산밖에 알지못하네요
141186,train,2-17419-2,11285.0,너무 많아서 하나하나 말씀드리가 어렵네요 ! 한번 찾아보시는걸 추천드려요 마음에 드...,32568.0,저는 보고나서 여운이 남는 영화는 별로 안 좋아해요. 코미디나 해피엔딩으로 끝나는 ...
141187,train,2-17419-2,11285.0,저는 액션을 좋아합니다 긴장감이돌고 심장이 쫄깃해져서 집중을 몰두하는편이에요 ! ...,32568.0,저는 정해진 주기로는 안가고 관심있는 영화가 있을때만 갑니다. 주로 OTT로 영화보...


In [8]:
from keybert import KeyBERT

def bert_keword(doc):
    model = KeyBERT('distilbert-base-nli-mean-tokens')
    keywords = model.extract_keywords(doc, stop_words=None)
    return keywords

In [16]:
tmp = list(train_df.loc[:20]['query'])

In [17]:
tmp

['안녕하세요. 저는 30대 남성입니다.',
 '반가워요 저는 선박정비원으로 일하고있습니다. 혹시 직업이 어떻게되시나요?',
 '군사학과는 주로 무슨 직업으로 가는건가요?',
 '저는 군대 때 운전병으로 나왔는데 운전 보직 좋은 것 같아요. ',
 '저는 삼겹살을 좋아합니다. 점심시간인데 점심은 드셧나요?',
 '지금 환절기 시즌이라 감기조심해야해요. 얇게 여러개를 입어야 할 것 같아요.',
 '감기 조심 하셔야 해요. 한번 걸리면 한주가 너무 힘든것 같아요.',
 '어제 업무는 잘 해결하셨나요? 27시간만이네요. 저는 내일 아울렛을 놀러갈까해요',
 '맞아요. 님은 어떤 옷을 주로 입으시나요?',
 '저는 맨투맨이나 면티 또는 셔츠를 좋아해요. 깔끔하게 입는게 좋은거같더라구요',
 '아하 맞아요 친구들 따라 다니다보며는 저도 지쳐서 못다니겟더라구요.',
 '맞아요.저도 카페에서 조금 쉬다가 간다하구 보고오라고한적도있어요',
 '저도 신발을 좋아해서 최근에만 3켤래나 구매했네요.',
 '저는 요번에 운동화를 샀어요! 다이어트 떄문에 런닝화 위주로 샀습니다.',
 '안녕하세요? 저는 10대 남자입니다!',
 '반가워요 ㅎㅎ 저는 동생이 초등학생일만큼 아직 어립니다 ㅋㅋ',
 '그렇습니다 ㅎㅎ 그런데 저는 요즘 가끔 외롭습니다 ㅠㅠ 이유를 아시나요?',
 '제가 노래방에 가는게 괴로워서 그렇습니다.. 친구들은 좋아하거든요. 그래서 소외감 느끼는데 어쩌죠?',
 '저는 피자 즐겨 먹어요! 혹시 좋아하나요?',
 '정말요? 그러면 어떤 피자를 드셔보셨나요? 추천부탁드립니다 ㅎㅎ',
 '와 정말 맛있을것 같아요 ㄷㄷ 저는 카페는 안 좋은 추억이 있는게, 디스코 음악을 싫어하는데 나오더라고요..']

In [18]:
from keybert import KeyBERT

def bert_keword(doc):
    model = KeyBERT('distilbert-base-nli-mean-tokens')
    keywords = model.extract_keywords(doc, stop_words=None)
    return keywords

In [21]:
data_lst = []
for i in tqdm(tmp):
    print(bert_keword(i))


  5%|▍         | 1/21 [00:00<00:13,  1.45it/s]

[('안녕하세요', 0.8835), ('남성입니다', 0.8823), ('저는', 0.7388), ('30대', 0.6597)]


 10%|▉         | 2/21 [00:01<00:13,  1.43it/s]

[('선박정비원으로', 0.946), ('반가워요', 0.8707), ('직업이', 0.8176), ('혹시', 0.7459), ('저는', 0.7413)]


 14%|█▍        | 3/21 [00:02<00:15,  1.16it/s]

...

[('먹어요', 0.8123), ('즐겨', 0.7979), ('저는', 0.7966), ('혹시', 0.7896), ('피자', 0.7508)]


 95%|█████████▌| 20/21 [00:15<00:00,  1.40it/s]

[('추천부탁드립니다', 0.9501), ('정말요', 0.8397), ('피자를', 0.8084), ('그러면', 0.7952), ('어떤', 0.2105)]


100%|██████████| 21/21 [00:16<00:00,  1.31it/s]

[('나오더라고요', 0.8521), ('카페는', 0.7999), ('디스코', 0.7993), ('음악을', 0.7794), ('정말', 0.7643)]


In [27]:
query_lst = list(train_df['query'])

In [ ]:
query_keword = [] 

for i in tqdm(query_lst):
    x = bert_keword(i)
    query_lst.append(x)

  8%|▊         | 11800/141189 [2:20:36<24:59:38,  1.44it/s]